In [ ]:
# Libraries to install
# !pip install torchsummary torchviz


  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4132 sha256=a0ee1421301099a58507a0a4c06ce39e55b062d3deafd4ecd9ac4feafe78e9f4
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [ ]:
# !pip install torchinfo


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

FILE_NAME = "poli_bias_distilbert"
PRETRAINED_MODEL = "dccuchile/distilbert-base-spanish-uncased"

# Model definition
class LibertyPredictor(nn.Module):
    def __init__(self, pretrained_model_name):
        super(LibertyPredictor, self).__init__()
        self.distilbert = AutoModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc_personal = nn.Linear(self.distilbert.config.hidden_size, 3)
        self.fc_economic = nn.Linear(self.distilbert.config.hidden_size, 3)

    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        x = self.dropout(pooled_output)
        personal_liberty = self.fc_personal(x)
        economic_liberty = self.fc_economic(x)
        return personal_liberty, economic_liberty

# Hyperparameters
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-04

# Tokenizer and model initialization
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LibertyPredictor(PRETRAINED_MODEL).to(device)

# Freeze DistilBERT layers except the last two
for name, param in model.distilbert.named_parameters():
    if 'layer.4' in name or 'layer.5' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Optimizer and loss function
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/530 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/269M [00:00<?, ?B/s]

In [ ]:
data = pd.read_csv('Manifesto_corpus_estructurado.csv')

data.head()

,Text,Code,libertad_personal_score,libertad_economica_score
0,DECLARACIÓN DE PRINCIPIOSFRENTE PARA LA VICTORIA,NaN,0,0
1,"Argentina, convicción y capacidad para constru...",NaN,0,0
2,La crisis ha sido el rostro de nuestro país du...,000,0,0
3,"Crisis económica, social, institucional, polít...",000,0,0
4,Sin reglas de juego claras y justas en lo econ...,503,1,-1


In [ ]:
data = data.drop("Code", axis = 1)

In [ ]:
data.head(8)

,Text,libertad_personal_score,libertad_economica_score
0,DECLARACIÓN DE PRINCIPIOSFRENTE PARA LA VICTORIA,0,0
1,"Argentina, convicción y capacidad para constru...",0,0
2,La crisis ha sido el rostro de nuestro país du...,0,0
3,"Crisis económica, social, institucional, polít...",0,0
4,Sin reglas de juego claras y justas en lo econ...,1,-1
5,La profundización hasta limites intolerables d...,1,-1
6,nos obliga a dirigir una mirada critica a quie...,0,0
7,La falta de capacidad en la administración de ...,0,0


In [ ]:
print(model)

LibertyPredictor(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): 

In [ ]:
import torch
from torchinfo import summary

# Asumiendo que su modelo se llama 'model' y ya está definido

# Crear entradas de ejemplo
input_ids = torch.zeros((1, 128), dtype=torch.long).to(device)
attention_mask = torch.ones((1, 128), dtype=torch.long).to(device)


In [ ]:
def generate_model_description(model):
    description = f"Descripción detallada de la arquitectura del modelo {type(model).__name__}:\n\n"

    for name, module in model.named_children():
        description += f"Capa: {name}\n"
        description += f"Tipo: {type(module).__name__}\n"
        if hasattr(module, 'in_features') and hasattr(module, 'out_features'):
            description += f"Dimensiones: {module.in_features} -> {module.out_features}\n"
        description += f"Parámetros entrenables: {sum(p.numel() for p in module.parameters() if p.requires_grad)}\n"
        description += "\n"

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    description += f"Total de parámetros: {total_params}\n"
    description += f"Parámetros entrenables: {trainable_params}\n"
    description += f"Parámetros no entrenables: {total_params - trainable_params}\n"

    return description

# Uso de la función
print(generate_model_description(model))

Descripción detallada de la arquitectura del modelo LibertyPredictor:

Capa: distilbert
Tipo: DistilBertModel
Parámetros entrenables: 14175744

Capa: dropout
Tipo: Dropout
Parámetros entrenables: 0

Capa: fc_personal
Tipo: Linear
Dimensiones: 768 -> 3
Parámetros entrenables: 2307

Capa: fc_economic
Tipo: Linear
Dimensiones: 768 -> 3
Parámetros entrenables: 2307

Total de parámetros: 66736134
Parámetros entrenables: 14180358
Parámetros no entrenables: 52555776



In [ ]:
def model_summary(model):
    print(model)
    print(f"\nNumber of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

# Llamar al resumen del modelo
model_summary(model)

LibertyPredictor(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): 